In [2]:
import sys, os
sys.path.append(os.getcwd())
sys.path.append('../..')
from imports import ilpgraph, readFile
from network import Steiner_Linear, Steiner_Linear_tightened
import networkx as nx
from gurobipy import *
import xlsxwriter

def __write_solution(solution:list):
    
    workbook = xlsxwriter.Workbook('Benchmarks_SteinFlow.xlsx', {'nan_inf_to_errors': True})
    worksheet = workbook.add_worksheet()
    bold = workbook.add_format({'bold':True})
    solutionRows = len(solution)
    solutionColumns = len(solution[0])
    
    worksheet.write('A1', 'Instance', bold)
    worksheet.write('B1', 'Edges', bold)
    worksheet.write('C1', 'Nodes', bold)
    worksheet.write('D1', 'LinearModel', bold)
    worksheet.write('G1', 'Constraints', bold)
    worksheet.write('H1', 'MIP Gap', bold)
    worksheet.write('E1', 'Runtime', bold)
    worksheet.write('D1', 'Tightened', bold)
    worksheet.write('G1', 'Constraints', bold)
    worksheet.write('H1', 'MIP Gap', bold)
    worksheet.write('E1', 'Runtime', bold)

    for i in range(solutionRows):
        for j in range(solutionColumns):
            worksheet.write(i + 1, j, solution[i][j])
    workbook.close()
    print("Finished!")
    
    
def runBenchmark(paths:str, linesToRead:range):
    solutions = []
    lines = []
    for path in paths:
        for i in range(len(linesToRead)):
            G, terminals = readFile.stp_to_networkx(path)
            Graph = ilpgraph.ILPGraph()
            Graph.setNXGraph(G)
            Graph.setEdgeVars(G.edges())
            Graph.setNodeVars(G.nodes())
            
            modelLin = Steiner_Linear.createModel(Graph, terminals)
            modelLin.Params.threads = 3
            modelLin.params.TimeLimit = 600
            modelLin.optimize() 

            modelTight = Steiner_Linear_tightened.createModel(Graph, terminals)
            modelTight.Params.threads = 3
            modelTight.params.TimeLimit = 600
            modelTight.optimize() 
            modelTight.optimize()
                
                
            solutions.append((i, len(G.edges()), len(G.nodes()), \
                modelLin.objVal, modelLin.numConstrs, modelLin.MIPGap, modelLin.runTime, \
                modelTight.objVal, modelTight.numConstrs, modelTight.MIPGap, modelTight.runTime))
        print(solutions)
    __write_solution(solutions)  
    

if __name__ == '__main__':
    linesToRead = range(0,1)
    
    paths = [r'..\\Steiner_testInstances\\d09.stp',
            r'..\\Steiner_testInstances\\d13.stp',
            r'..\\Steiner_testInstances\\e08.stp',
            r'..\\Steiner_testInstances\\e10.stp',
            r'..\\Steiner_testInstances\\e13.stp',
            r'..\\Steiner_testInstances\\e15.stp',
            r'..\\Steiner_testInstances\\e16.stp',
            r'..\\Steiner_testInstances\\e17.stp',
            r'..\\Steiner_testInstances\\e18.stp',
            r'..\\Steiner_testInstances\\e19.stp',
            r'..\\Steiner_testInstances\\e20.stp',]

    runBenchmark(paths, linesToRead)
    





--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2021-03-23
Using license file C:\Program Files\Gurobi\gurobi.lic
Changed value of parameter MIPGap to 0.05
   Prev: 0.0001  Min: 0.0  Max: inf  Default: 0.0001
Changed value of parameter threads to 3
   Prev: 0  Min: 0  Max: 1024  Default: 0
Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 3 threads
Optimize a model with 18251 rows, 6000 columns and 70250 nonzeros
Model fingerprint: 0x1a3429e3
Variable types: 0 continuous, 6000 integer (5000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+03]
  RHS range        [1e+00, 1e+03]
Presolve removed 9354 rows and 330 columns
Presolve time: 0.20s
Presolve

TypeError: NAN/INF not supported in write_number() without 'nan_inf_to_errors' Workbook() option